# Dataset Statistics

In [ ]:
# libraries

In [ ]:
# common functions

In [ ]:
def tokenizer():
    pass

def count_tokens():
    pass
def compute_stats():
    pass

In [ ]:
def visualize_stats_with_barchart():
    pass

## TACRED

In [ ]:
# train_path
# val_path
# test_path

## FewRel

In [ ]:
# train_path
# val_path
# test_path
